<a href="https://colab.research.google.com/github/screemix/The_Glorious_Chargers/blob/main/Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import glob
import os
import io
import pickle
import statistics
import math
import pandas as pd
import numpy as np

import seaborn as sns

import calendar
calendar.setfirstweekday(calendar.MONDAY) # first week day

from datetime import datetime
from datetime import time
from datetime import date, timedelta

from dateutil.relativedelta import *

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from pandas.plotting import autocorrelation_plot
from pandas.plotting import lag_plot
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from pandas.tseries.offsets import *

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import seasonal_decompose


%matplotlib inline

# pandas settings
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, cross_val_score, \
                                    ShuffleSplit, cross_val_predict, TimeSeriesSplit, ParameterSampler, PredefinedSplit
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, PolynomialFeatures, LabelEncoder, OneHotEncoder, LabelBinarizer
from sklearn.metrics import r2_score,  mean_squared_error, accuracy_score, \
                            mean_absolute_error, median_absolute_error, make_scorer
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LogisticRegression, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.dummy import DummyRegressor
from sklearn.feature_selection import RFE, RFECV, SelectFromModel

import xgboost as xgb
from xgboost import plot_importance

In [3]:
pip install fsspec

     |████████████████████████████████| 92kB 3.5MB/s 


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Data upload

In [9]:
from google.colab import files
uploaded = files.upload()

Saving train_v2.csv to train_v2 (1).csv


In [10]:
train_v2_ = pd.read_csv('/content/My Drive/train_v2.csv')

In [13]:
train_v2 = pd.read_csv(io.BytesIO(uploaded['train_v2.csv']))
train_v2.head()

,phone_id,months_with_order,months_without_order,target,gender,age
0,2,3,4,0,NaN,NaN
1,3,3,4,0,NaN,NaN
2,4,4,3,0,NaN,NaN
3,6,4,3,0,NaN,NaN
4,6,4,3,0,NaN,NaN


Train-test split (Katya)

In [ ]:
train_and_test_y = data['Target']

NameError: ignored

Simple Models

In [ ]:
gbm = GradientBoostingClassifier(random_state=42)
xg = xgb.XGBClassifier(random_state=420)

models = {
    'Decision Tree': {'base_model': dt},
    'Random Forest': {'base_model': rf},    
    'GBR': {'base_model': gbr},
    'XGBoost': {'base_model': xg}
}

NameError: ignored

In [ ]:
results = {}
# Appy each model on the test set
for model_name, model in models.items():
    
    #scale test set here
    
    y_pred = model[use_model].predict(test_X) # Prediction
   
    mse = mean_squared_error(test_y, y_pred)  # Measures
    rmse = sqrt(mean_squared_error(test_y, y_pred))
    mae = mean_absolute_error(test_y, y_pred)
    r2 = r2_score(test_y, y_pred)
    abs_error = abs(test_y.values - y_pred)
 
    results[model_name] = {'Model': model, 
                              'Absolute error': abs_error,
                              'R2':r2, 
                              'MSE': mse, 
                              'RMSE': rmse,
                              'MAE': mae
                             }
# Summary of the results
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df

In [ ]:
for model_name, model in models.items():
    print('Training {}...\n'.format(model_name))
    model[use_model].fit(train_X_antrag, train_y)
    
    importances = model[use_model].feature_importances_
    indices = np.argsort(importances)[::-1]
    
    train_X.columns[indices]
    
    i = 1
    for idx in indices:
        if importances[idx] > 0.01:
            print('{}. {} - {}'.format(i, train_X.columns[idx], round(importances[idx], 3)))
        i+=1
    print('\n')

For model-specific hyperparameter tuning

In [ ]:
mod_GBM = GradientBoostingClassifier(random_state=42)

search_grid_GBM = {'learning_rate': [0.05],
                   'n_estimators': [50],
                   'max_depth': [4],
                   'min_samples_leaf': [10,25,50],
                   'min_samples_split': [25,50],
                   'max_features': ['sqrt'],
                   'subsample': [0.8],
                   'random_state': [29]
                   }

In [ ]:
param_search_GBM = GridSearchCV(estimator=mod_GBM, 
                                param_grid=search_grid_GBM, 
                                scoring=metrics.make_scorer(metrics.roc_auc_score),
                                cv=3, 
                                n_jobs=-1, 
                                verbose=2,
                                return_train_score=True,
                                iid=True)

param_search_GBM.fit(X_train, y_train)

print('Optimal parameters are:')
print(param_search_GBM.best_params_)

In [ ]:
#### Creation of the model object with optimal hyperparameters

In [ ]:
mod_opt_GBM = GradientBoostingClassifier(**param_search_GBM.best_params_)
mod_opt_GBM = mod_opt_GBM.fit(X_train, y_train)

In [ ]:
# Distribution of scores by class
score_train_GBM = mod_opt_GBM.predict_proba(X_train)[:, 1]
score_test_GBM = mod_opt_GBM.predict_proba(X_test)[:, 1]

fig, ax = plt.subplots(ncols=2, figsize=(15, 5))

(pd.DataFrame({'y': y_train, 'Training score': score_train_GBM})
 .boxplot(column='Training score', by='y',
          showfliers=True, ax=ax[0]))
plt.title('Training score')

(pd.DataFrame({'y': y_test, 'Test score': score_test_GBM})
 .boxplot(column='Test score', by='y', showfliers=True, ax=ax[1]))
plt.title('Test score')

plt.suptitle('Distribution of scores by class')

plt.show()

In [ ]:
# ROC Curve
fpr_train_GBM, tpr_train_GBM, thresholds_train_GBM = \
    metrics.roc_curve(y_train, score_train_GBM, pos_label=1)

fpr_test_GBM, tpr_test_GBM, thresholds_test_GBM = \
    metrics.roc_curve(y_test, score_test_GBM, pos_label=1)

fig = plt.figure(figsize=(10, 6))
plt.plot(fpr_train_GBM, tpr_train_GBM, label='training data')
plt.plot(fpr_test_GBM, tpr_test_GBM, label='test data')
plt.legend()
plt.title('ROC-curve for GBM')
plt.show()